In [1]:
import pandas as pd
from datetime import datetime
from tokenizers import ByteLevelBPETokenizer
from tqdm.auto import tqdm

from llm_lang.utils import get_dataset, get_data_column, get_tokenizer

# # Change the date to the current date if you want to run the script
# DATE = datetime.now().strftime("%Y-%m-%d")
# # Load the dataset for a specific run date
DATE = "2024-08-31"
base_dir = "../data/processed/translate-batch/" + DATE + "/"

In [2]:
ds = get_dataset(lang="all")

# Ensure that all languages have the same number of sentences
lens = [len(ds[col]) for col in ds.column_names if col.startswith("sentence_")]

assert min(lens) == max(lens)

# LLM performance measurement


We proxy the measurment of an LLMs performance using the back-translation task.

We will use also an LLM to quantify the quality of the back-translation.

Method:

- Select N target languages.
- Collect the sentence pairs from the FLORES dataset for the selected languages.
- Ask the LLM to translate the non-English sentences into English.
- Take the original English sentence and the LLM-translated English sentence and ask an LLM to qualify whether the two sentences convey exactly the same meaning.

In [3]:
# lang_name = dict(
#     shn="Shan",
#     sat="Santali",
#     tel="Telugu",
#     amh="Amharic",
#     ben="Bengali",
#     hin="Hindi",
#     swa="Swahili",
#     swh="Swahili",
#     zho="Chinese",
#     arb="Arabic",
#     jpn="Japanese",
#     deu="German",
#     spa="Spanish",
#     por="Portuguese",
#     eng="English",
#     tgl="Tagalog",
# )

## Shortlist languages  # Check the analysis.ipynb notebook for the details
lang_shortlist = [
    # - Top 3 languages with the highest premium from low-income countries.
    dict(
        dzo=dict(
            name="Dzongkha",
            code="dzo_Tibt",
        ),
        taq=dict(
            name="Tamasheq",
            code="taq_Latn",
        ),
        kbp=dict(
            name="Kabiyè",
            code="kbp_Latn",
        ),
    ),
    # - Top 3 languages with the highest premium from low-middle-income countries.
    dict(
        shn=dict(
            name="Shan",
            code="shn_Mymr",
        ),
        sat=dict(
            name="Santhali",
            code="sat_Olck",
        ),
        ory=dict(
            name="Odia",
            code="ory_Orya",
        ),
    ),
    # - Top 3 languages by total population with at least 4x premium from low-income countries.
    dict(
    nus=dict(
        name="Nuer",
        code="nus_Latn",
    ),
    kbp=dict(
        name="Kabiyè",
        code="kbp_Latn",
    ),
    taq=dict(
        name="Tamasheq",
        code="taq_Latn",
    ),
    ),
    # - Top 3 languages by total population with at least 4x premium from low-middle-income countries.
    dict(
        hin=dict(
            name="Hindi",
            code="hin_Deva",
        ),
        ben=dict(
            name="Bengali",
            code="ben_Beng",
        ),
        urd=dict(
            name="Urdu",
            code="urd_Arab",
        ),
    ),
    # - Top 5 languages by total population.
    dict(
        eng=dict(
            name="English",
            code="eng_Latn",
        ),
        zho=dict(
            name="Chinese",
            code="zho_Hans",
        ),
        hin=dict(
            name="Hindi",
            code="hin_Deva",
        ),
        spa=dict(
            name="Spanish",
            code="spa_Latn",
        ),
        arb=dict(
            name="Standard Arabic",
            code="arb_Arab",
        ),
        fra=dict(
            name="French",
            code="fra_Latn",
        ),
    ),
]

ordered_lang_names = ["nus", "kbp", "taq", "dzo", "hin", "ben", "urd", "ory", "sat", "shn", "zho", "arb", "spa", "fra"]

lang_name = {k: v for d in lang_shortlist for k, v in d.items()}
lang_name

{'dzo': {'name': 'Dzongkha', 'code': 'dzo_Tibt'},
 'taq': {'name': 'Tamasheq', 'code': 'taq_Latn'},
 'kbp': {'name': 'Kabiyè', 'code': 'kbp_Latn'},
 'shn': {'name': 'Shan', 'code': 'shn_Mymr'},
 'sat': {'name': 'Santhali', 'code': 'sat_Olck'},
 'ory': {'name': 'Odia', 'code': 'ory_Orya'},
 'nus': {'name': 'Nuer', 'code': 'nus_Latn'},
 'hin': {'name': 'Hindi', 'code': 'hin_Deva'},
 'ben': {'name': 'Bengali', 'code': 'ben_Beng'},
 'urd': {'name': 'Urdu', 'code': 'urd_Arab'},
 'eng': {'name': 'English', 'code': 'eng_Latn'},
 'zho': {'name': 'Chinese', 'code': 'zho_Hans'},
 'spa': {'name': 'Spanish', 'code': 'spa_Latn'},
 'arb': {'name': 'Standard Arabic', 'code': 'arb_Arab'},
 'fra': {'name': 'French', 'code': 'fra_Latn'}}

In [4]:
lang_index = pd.DataFrame(
    [(lang_name[name]["name"], name) for name in ordered_lang_names], columns=["Language", "Code"],
    index=[lang_name[name]["code"] for name in ordered_lang_names]
)
lang_index

,Language,Code
nus_Latn,Nuer,nus
kbp_Latn,Kabiyè,kbp
taq_Latn,Tamasheq,taq
dzo_Tibt,Dzongkha,dzo
hin_Deva,Hindi,hin
ben_Beng,Bengali,ben
urd_Arab,Urdu,urd
ory_Orya,Odia,ory
sat_Olck,Santhali,sat
shn_Mymr,Shan,shn


In [5]:
data = get_data_column(ds, "all")
data_eng = get_data_column(ds, "eng_Latn")
data_hin = get_data_column(ds, "hin_Deva")
data_tgl = get_data_column(ds, "tgl_Latn")
data_sat = get_data_column(ds, "sat_Olck")  # Santhali
data_tel = get_data_column(ds, "tel_Telu")  # Telugu
data_amh = get_data_column(ds, "amh_Ethi")  # Amharic

gpt4_tokenizer = get_tokenizer("gpt-4")
gpt4o_tokenizer = get_tokenizer("gpt-4o")

models = ["gpt-4-turbo-2024-04-09", "gpt-4o"]  # , "gpt-4"]

In [6]:
from hashlib import md5


def build_batch_translate_to_english(lang_src, model_kwargs):
    lang_tgt = "eng_Latn"

    input_sentences = get_data_column(ds, lang_src)
    target_sentences = get_data_column(ds, lang_tgt)

    lang_src_name = lang_src.split("_")[0]
    lang_tgt_name = lang_tgt.split("_")[0]

    system_message = dict(
        # content=f"You are a highly capable machine translation system that translates {lang_name[lang_src_name]['name']} to {lang_name[lang_tgt_name]['name']}. Your response must start with `{lang_name[lang_tgt_name]['name']}: ` followed by the translation.",
        content=f"""You are a highly advanced machine translation system specializing in translations from {lang_name[lang_src_name]['name']} to {lang_name[lang_tgt_name]['name']}. Please translate the given text by the user, and format your response as follows: `{lang_name[lang_tgt_name]['name']}: <translation>`.\n\nProvide a high-quality translation that accurately conveys the meaning of the original text.""",
        role="system"
    )
    prompt_id = md5(system_message["content"].encode()).hexdigest()
    payloads = []

    for idx, (sent_src, sent_tgt) in enumerate(zip(input_sentences, target_sentences)):
        # hint_token = tokenizer.decode([tokenizer.encode(sent_tgt)[0]])
        # user_message = f"{sent_src}\n\n{hint_token}"
        # user_message = f"{lang_name[lang_src_name]['name']}: {sent_src}\n\n{lang_name[lang_tgt_name]['name']}: "
        user_message = f"{lang_name[lang_src_name]['name']}: {sent_src}\n"

        messages = [
            system_message,
            dict(
                content=user_message,
                role="user"
            )
        ]

        payloads.append(
            dict(
                custom_id=f"{lang_src}-{lang_tgt}-{idx:03d}-{model_kwargs['model']}-{prompt_id}",
                method="POST",
                url="/v1/chat/completions",
                body=dict(**model_kwargs, messages=messages))
        )

    return dict(payloads=payloads, prompt_id=prompt_id)

In [7]:
model_kwargs = dict(
    model="gpt-4o",
    seed=1029,
    temperature=0,
    max_tokens=4096,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    logprobs=True,
)
# payloads = build_batch_translate_to_english("hin_Deva", model_kwargs)

In [21]:
# Generate the batch files for each language of interest
import json
import os

batch_files = []

for gpt_model in ["gpt-4-turbo-2024-04-09", "gpt-4o"]:
    model_kwargs = dict(
        model=gpt_model,
        seed=1029,
        temperature=0,
        max_tokens=4096,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        logprobs=True,
    )

    for lang_src in lang_name:
        lang_src = lang_name[lang_src]["code"]

        if lang_src == "eng_Latn":
            continue

        output = build_batch_translate_to_english(lang_src, model_kwargs)
        payloads = output["payloads"]
        prompt_id = output["prompt_id"]
        pathname = f"batch-{lang_src}-{model_kwargs['model']}-{prompt_id}.jsonl"
        pathname = os.path.join(base_dir, pathname)

        os.makedirs(os.path.dirname(pathname), exist_ok=True)

        if os.path.exists(pathname):
            batch_files.append(pathname)
            continue

        with open(pathname, "w") as f:
            for payload in payloads:
                f.write(json.dumps(payload) + "\n")

        batch_files.append(pathname)

batch_files

['../data/processed/translate-batch/2024-08-31/batch-dzo_Tibt-gpt-4-turbo-2024-04-09-eb35f392639e8f74bae35d15fe673b2a.jsonl',
 '../data/processed/translate-batch/2024-08-31/batch-taq_Latn-gpt-4-turbo-2024-04-09-718c226fdacdb5cf3e9b69d45017db10.jsonl',
 '../data/processed/translate-batch/2024-08-31/batch-kbp_Latn-gpt-4-turbo-2024-04-09-c15992ca5c7b045c59118814cf1e9612.jsonl',
 '../data/processed/translate-batch/2024-08-31/batch-shn_Mymr-gpt-4-turbo-2024-04-09-c316d49a84b79a00db60e597a916ad9c.jsonl',
 '../data/processed/translate-batch/2024-08-31/batch-sat_Olck-gpt-4-turbo-2024-04-09-fd8a44d68276a4ab4dd014248d364da6.jsonl',
 '../data/processed/translate-batch/2024-08-31/batch-ory_Orya-gpt-4-turbo-2024-04-09-a5ca1a11b916f7eaaf80b561bd41674e.jsonl',
 '../data/processed/translate-batch/2024-08-31/batch-nus_Latn-gpt-4-turbo-2024-04-09-abe6494914969665b513a6a18abc27c0.jsonl',
 '../data/processed/translate-batch/2024-08-31/batch-hin_Deva-gpt-4-turbo-2024-04-09-aa9307c7810a51af3e2000e4ff3c95a5.

## Submit files to OpenAI

In [22]:
from openai import OpenAI
client = OpenAI()

batch_input_files = []

for pathname in batch_files:
    translated_pathname = pathname.replace(".jsonl", "-translated.jsonl")
    if os.path.exists(translated_pathname):
        continue

    batch_input_file = client.files.create(
        file=open(pathname, "rb"),
        purpose="batch"
    )
    batch_input_files.append(batch_input_file)

## Create the batch runs

In [24]:
batch_objects = []

for batch_input_file in batch_input_files:
    batch_input_file_id = batch_input_file.id

    print(batch_input_file_id)

    bobj = client.batches.create(
        input_file_id=batch_input_file_id,
        endpoint="/v1/chat/completions",
        completion_window="24h",
        metadata={
        "description": f"translate job for {batch_input_file.filename}",
        }
    )

    batch_objects.append(bobj)

file-7gUZ746yKACj98aH6xyHBg6I
file-hgRmeqmP0gOmtjHpF9eAi3L9
file-I4pTWWOEqTodsVUWj8lQadYK
file-x6HnIl9tlBsyVMVrdcJYkP2I
file-2ZsjKwgp6oRKe9fRdHSV69do


## Track and save the results

In [25]:
# Check the status of the batch
for batch_object in batch_objects:
    status = client.batches.retrieve(batch_object.id)
    print(status.status, status.metadata)

    if status.status == "completed":
        fname = status.metadata["description"].split(" ")[-1].replace(".jsonl", "-translated.jsonl")
        fname = os.path.join(base_dir, fname)

        if os.path.exists(fname):
            continue

        content = client.files.content(status.output_file_id)

        with open(fname, "wb") as f:
            f.write(content.content)

completed {'description': 'translate job for batch-dzo_Tibt-gpt-4-turbo-2024-04-09-eb35f392639e8f74bae35d15fe673b2a.jsonl'}
completed {'description': 'translate job for batch-taq_Latn-gpt-4-turbo-2024-04-09-718c226fdacdb5cf3e9b69d45017db10.jsonl'}
completed {'description': 'translate job for batch-kbp_Latn-gpt-4-turbo-2024-04-09-c15992ca5c7b045c59118814cf1e9612.jsonl'}
completed {'description': 'translate job for batch-shn_Mymr-gpt-4-turbo-2024-04-09-c316d49a84b79a00db60e597a916ad9c.jsonl'}
completed {'description': 'translate job for batch-sat_Olck-gpt-4-turbo-2024-04-09-fd8a44d68276a4ab4dd014248d364da6.jsonl'}


## Create comparison payloads

In [8]:
def build_comparison_payload(model_kwargs, system_content: str = None):
    assert model_kwargs["model"]  == "gpt-4o", "Only GPT-4o model is supported for comparison"

    # List all the translated files
    translated_files = sorted([os.path.join(base_dir, fname) for fname in os.listdir(base_dir) if fname.endswith("-translated.jsonl")])

    if system_content is None:
        # system_content = "You are a highly capable machine translation system so you can judge the quality of a translation.\n\nGiven a source text, rate the translation quality that produced the translated text using a five-point scale: Poor, Fair, Good, Very Good, and Excellent.\n\nYour response must start with `Rating: ` followed by your rating. Do not explain."

        system_content = """You are an expert machine translation evaluation system, capable of accurately assessing translation quality. Given a source text and its translated counterpart, rate the translation quality using a 5-point scale: Poor, Fair, Good, Very Good, Excellent. The scale is defined as follows:\n\n**Poor**: The translation is barely comprehensible, contains significant errors, and may not convey the original message. It may require extensive editing or retranslation.\n\n**Fair**: The translation is understandable but contains noticeable errors, inaccuracies, or awkward phrasing. It may require some editing to improve clarity and accuracy.\n\n**Good**: The translation is generally accurate and clear, but may contain minor errors or slight inaccuracies. It is suitable for general use but may not be perfect for critical or high-stakes applications.\n\n**Very Good**: The translation is highly accurate, clear, and nuanced, with only minor imperfections. It is suitable for most professional purposes and demonstrates a strong understanding of the source text.\n\n**Excellent**: The translation is virtually flawless, conveying the exact meaning, tone, and nuance of the original text. It is suitable for high-stakes applications, such as official publications or critical communications.\n\nFirst, explain to yourself in one sentence the reason for your rating. Then, end your response with `Rating: <rating>`."""

    system_message = dict(
        content=system_content,
        role="system"
    )

    prompt_id = md5(system_message["content"].encode()).hexdigest()
    comparison_fname = os.path.join(base_dir, "comparison", f"comparison-{prompt_id}.jsonl")

    completed_fname = os.path.join(base_dir, "comparison", f"comparison-{prompt_id}-completed.jsonl")

    assert not os.path.exists(completed_fname), "The comparison file has already been completed. Please check the file."

    completed_data = {}

    # if os.path.exists(completed_fname):
    #     comparison_fname = comparison_fname.replace(".jsonl", "-v1.jsonl")
    #     # Load data from the comparison file to use in filtering
    #     # those that have already been completed.
    #     with open(completed_fname, "r") as f:
    #         completed_data = [json.loads(line) for line in f]
    #         completed_data = {data["custom_id"].replace(f"comparison-{prompt_id}-", ""): data for data in completed_data}

    print(f"Completed data: {len(completed_data)}")
    payloads = []

    for fname in tqdm(translated_files):
        # print(fname)

        with open(fname, "r") as f:
            responses = [json.loads(line) for idx, line in enumerate(f)]

        for data in sorted(responses, key=lambda x: x["custom_id"]):
            custom_id = data["custom_id"]

            if custom_id in completed_data:
                continue

            fragments = custom_id.split("-")
            idx = int(fragments[2])

            eng_sentence = data_eng[idx]
            translated = data["response"]["body"]["choices"][0]["message"]["content"]

            assert translated.startswith("English: ")
            translated = translated.replace("English: ", "")

            user_message = f"Source: {eng_sentence}\n\nTranslated: {translated}\n"
            messages = [
                system_message,
                dict(
                    content=user_message,
                    role="user"
                )
            ]

            payloads.append(
                dict(
                    custom_id=f"comparison-{prompt_id}-{custom_id}",
                    method="POST",
                    url="/v1/chat/completions",
                    body=dict(**model_kwargs, messages=messages))
            )

    assert len(payloads) == (997 * (len(lang_name) - 1) * len(models)), len(payloads) # 997 is the number of sentences in each language except English  ## 27916

    os.makedirs(os.path.dirname(comparison_fname), exist_ok=True)

    with open(comparison_fname, "w") as f:
        for payload in payloads:
            f.write(json.dumps(payload) + "\n")

    return dict(payloads=payloads, prompt_id=prompt_id)

In [ ]:
comparison_data = build_comparison_payload(model_kwargs)

## Prepare comparison batch

In [32]:
comparison_fname = os.path.join(base_dir, "comparison", f"comparison-{comparison_data['prompt_id']}.jsonl")

comparison_batch_input_file = client.files.create(
    file=open(comparison_fname, "rb"),
    purpose="batch"
)

comparison_bobj = client.batches.create(
    input_file_id=comparison_batch_input_file.id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
    "description": f"comparison job for {comparison_batch_input_file.filename}",
    }
)

In [37]:
from datetime import datetime
import time

def check_store_comparison(comparison_bobj):

    status = client.batches.retrieve(comparison_bobj.id)

    while status.status != "completed":
        print(datetime.now(), status.status, status.request_counts, status.metadata)
        time.sleep(5 * 60)
        status = client.batches.retrieve(comparison_bobj.id)

    print(datetime.now(), status.status, status.request_counts, status.metadata)
    fname = status.metadata["description"].split(" ")[-1].replace(".jsonl", "-completed.jsonl")
    fname = os.path.join(base_dir, "comparison", fname)

    if not os.path.exists(fname):
        content = client.files.content(status.output_file_id)

        with open(fname, "wb") as f:
            f.write(content.content)

    return fname

fname = check_store_comparison(comparison_bobj)

2024-09-01 01:37:00.078290 completed BatchRequestCounts(completed=4985, failed=0, total=4985) {'description': 'comparison job for comparison-e1e67f099c5f74f0508c6305f749b372-v1.jsonl'}


## Analyze comparison results

In [9]:
## RESUME HERE

def parse_comparison_response(fname, valid_ratings = None):
    if valid_ratings is None:
        valid_ratings = ["INCORRECT", "CORRECT"]

    with open(fname, "r") as f:
        comparison_responses = [json.loads(line) for idx, line in enumerate(f)]

    comparisons = []

    for data in comparison_responses:
        fragments = data["custom_id"].split("-")
        idx = int(fragments[4])
        lang_src = fragments[2]
        model = "-".join(fragments[5:-1])

        rating = data["response"]["body"]["choices"][0]["message"]["content"]

        has_match = [f"Rating: {valid_rating}" in rating for valid_rating in valid_ratings]

        if sum(has_match) != 1:
            invalid_snippets = [
                "sentence is not provided",
                "provided translation is not in English",
            ]
            for invalid_snippet in invalid_snippets:
                if invalid_snippet in rating:
                    if "INCORRECT" in valid_ratings:
                        rating = "Rating: INCORRECT"
                    elif "Poor" in valid_ratings:
                        rating = "Rating: Poor"
                    else:
                        raise ValueError(f"Invalid rating: {rating}, {data['custom_id']}")
                    break
            else:
                raise ValueError(f"Invalid rating: {rating}, {data['custom_id']}")

        rating = rating[rating.find("Rating"):].replace("Rating: ", "")

        # assert ("Rating: CORRECT" in rating) or ("Rating: INCORRECT" in rating)

        # # assert rating.startswith("Rating: ")
        # # rating = rating.replace("Rating: ", "")
        # if "Rating: CORRECT" in rating:
        #     assert "Rating: INCORRECT" not in rating, "Rating should be either CORRECT or INCORRECT"
        #     rating = "CORRECT"
        # elif "Rating: INCORRECT" in rating:
        #     rating = "INCORRECT"
        # else:
        #     raise ValueError(f"Invalid rating: {rating}")

        comparisons.append(
            dict(
                idx=idx,
                lang_src=lang_src,
                model=model,
                rating=rating,
            )
        )

    return comparisons

In [14]:
import os
import json

fname = os.path.join(base_dir, "comparison", "comparison-e1e67f099c5f74f0508c6305f749b372-completed.jsonl")

parsed_comparisons = parse_comparison_response(
    fname,
    valid_ratings=["Poor", "Fair", "Good", "Very Good", "Excellent"]
)
parsed_comparisons_df = pd.DataFrame(parsed_comparisons)

performance_df = pd.DataFrame(parsed_comparisons)
performance_df = performance_df.groupby(["model", "lang_src", "rating"]).size().unstack(level=[0, 2])
performance_df /= performance_df["gpt-4o"].sum(axis=1).values[0]

# Sort the rating columns in increasing order
# ["Poor", "Fair", "Good", "Very Good", "Excellent"]
rating_order = ["Poor", "Fair", "Good", "Very Good", "Excellent"]
cols = [(m, r) for m in models for r in rating_order]

performance_df = performance_df[cols]
performance_df = (
    performance_df
    .loc[[lang_name[name]["code"] for name in ordered_lang_names]]
    .set_index(lang_index.set_index(["Language", "Code"]).index)
)
performance_df


model                gpt-4-turbo-2024-04-09                                \
rating                                 Poor      Fair      Good Very Good   
Language        Code                                                        
Nuer            nus                0.943831  0.031093  0.023069  0.002006   
Kabiyè          kbp                0.927783  0.036108  0.029087  0.004012   
Tamasheq        taq                0.890672  0.054162  0.044132  0.006018   
Dzongkha        dzo                0.978937  0.017051  0.004012       NaN   
Hindi           hin                0.036108  0.083250  0.422267  0.343029   
Bengali         ben                0.048144  0.105316  0.448345  0.277834   
Urdu            urd                0.046138  0.093280  0.418255  0.315948   
Odia            ory                0.096289  0.110331  0.416249  0.265797   
Santhali        sat                0.998997  0.001003       NaN       NaN   
Shan            shn                0.894684  0.049147  0.042126  0.010030   
Chinese         zho                0.011033  0.030090  0.320963  0.458375   
Standard Arabic arb                0.020060  0.047141  0.305918  0.499498   
Spanish         spa                0.007021  0.022066  0.275827  0.539619   
French          fra                0.007021  0.016048  0.183551  0.662989   

model                             gpt-4o                                \
rating               Excellent      Poor      Fair      Good Very Good   
Language        Code                                                     
Nuer            nus        NaN  0.939819  0.026078  0.029087  0.003009   
Kabiyè          kbp   0.003009  0.894684  0.053159  0.040120  0.006018   
Tamasheq        taq   0.005015  0.845537  0.077232  0.053159  0.017051   
Dzongkha        dzo        NaN  0.787362  0.119358  0.075226  0.011033   
Hindi           hin   0.115346  0.032096  0.063190  0.392177  0.367101   
Bengali         ben   0.120361  0.037111  0.092277  0.405216  0.342026   
Urdu            urd   0.126379  0.040120  0.083250  0.393180  0.359077   
Odia            ory   0.111334  0.054162  0.089268  0.371113  0.351053   
Santhali        sat        NaN  0.990973  0.006018  0.003009       NaN   
Shan            shn   0.004012  0.879639  0.059178  0.042126  0.013039   
Chinese         zho   0.179539  0.008024  0.025075  0.286861  0.499498   
Standard Arabic arb   0.127382  0.011033  0.023069  0.277834  0.548646   
Spanish         spa   0.155466  0.002006  0.025075  0.263791  0.549649   
French          fra   0.130391  0.006018  0.016048  0.160481  0.667001   

model                           
rating               Excellent  
Language        Code            
Nuer            nus   0.002006  
Kabiyè          kbp   0.006018  
Tamasheq        taq   0.007021  
Dzongkha        dzo   0.007021  
Hindi           hin   0.145436  
Bengali         ben   0.123370  
Urdu            urd   0.124373  
Odia            ory   0.134403  
Santhali        sat        NaN  
Shan            shn   0.006018  
Chinese         zho   0.180542  
Standard Arabic arb   0.139418  
Spanish         spa   0.159478  
French          fra   0.150451

In [142]:
performance_df = pd.DataFrame(parsed_comparisons)
performance_df = performance_df.groupby(["model", "lang_src", "rating"]).size().unstack(level=[0, 2])
performance_df /= performance_df["gpt-4"].sum(axis=1).values[0]
performance_df

model        gpt-4                                            gpt-4o  \
rating   Excellent      Fair      Good      Poor Very Good Excellent   
lang_src                                                               
amh_Ethi  0.088265  0.268806  0.089268  0.447342  0.106319  0.288867   
arb_Arab  0.539619  0.105316  0.120361  0.033099  0.201605  0.672016   
ben_Beng  0.262788  0.280843  0.147442  0.120361  0.188566  0.423270   
deu_Latn  0.780341  0.020060  0.041123  0.006018  0.152457  0.825476   
hin_Deva  0.362086  0.221665  0.159478  0.063190  0.193581  0.497492   
jpn_Jpan  0.409228  0.101304  0.134403  0.017051  0.338014  0.507523   
por_Latn  0.695085  0.046138  0.063190  0.008024  0.187563  0.747242   
sat_Olck       NaN  0.001003       NaN  0.998997       NaN       NaN   
spa_Latn  0.544634  0.058175  0.084253  0.010030  0.302909  0.633902   
swh_Latn  0.395186  0.183551  0.139418  0.064193  0.217653  0.527583   
tel_Telu  0.203611  0.292879  0.114343  0.231695  0.157472  0.468405   
tgl_Latn  0.562688  0.121364  0.105316  0.015045  0.195587  0.669007   
zho_Hans  0.441324  0.102307  0.126379  0.019057  0.310933  0.556670   

model                                             
rating        Fair      Good      Poor Very Good  
lang_src                                          
amh_Ethi  0.237713  0.135406  0.129388  0.208626  
arb_Arab  0.062187  0.071214  0.017051  0.177533  
ben_Beng  0.178536  0.127382  0.049147  0.221665  
deu_Latn  0.016048  0.029087  0.006018  0.123370  
hin_Deva  0.143430  0.115346  0.039117  0.204614  
jpn_Jpan  0.061184  0.118355  0.017051  0.295888  
por_Latn  0.044132  0.044132  0.007021  0.157472  
sat_Olck  0.010030  0.001003  0.988967       NaN  
spa_Latn  0.052156  0.066199  0.007021  0.240722  
swh_Latn  0.117352  0.111334  0.049147  0.194584  
tel_Telu  0.154463  0.121364  0.053159  0.202608  
tgl_Latn  0.087262  0.068205  0.009027  0.166499  
zho_Hans  0.056169  0.094283  0.012036  0.280843

In [143]:
# Sort the rating columns in increasing order
# ["Poor", "Fair", "Good", "Very Good", "Excellent"]
rating_order = ["Poor", "Fair", "Good", "Very Good", "Excellent"]
cols = [(m, r) for m in ["gpt-4", "gpt-4o"] for r in rating_order]

performance_df = performance_df[cols]
performance_df

model        gpt-4                                            gpt-4o  \
rating        Poor      Fair      Good Very Good Excellent      Poor   
lang_src                                                               
amh_Ethi  0.447342  0.268806  0.089268  0.106319  0.088265  0.129388   
arb_Arab  0.033099  0.105316  0.120361  0.201605  0.539619  0.017051   
ben_Beng  0.120361  0.280843  0.147442  0.188566  0.262788  0.049147   
deu_Latn  0.006018  0.020060  0.041123  0.152457  0.780341  0.006018   
hin_Deva  0.063190  0.221665  0.159478  0.193581  0.362086  0.039117   
jpn_Jpan  0.017051  0.101304  0.134403  0.338014  0.409228  0.017051   
por_Latn  0.008024  0.046138  0.063190  0.187563  0.695085  0.007021   
sat_Olck  0.998997  0.001003       NaN       NaN       NaN  0.988967   
spa_Latn  0.010030  0.058175  0.084253  0.302909  0.544634  0.007021   
swh_Latn  0.064193  0.183551  0.139418  0.217653  0.395186  0.049147   
tel_Telu  0.231695  0.292879  0.114343  0.157472  0.203611  0.053159   
tgl_Latn  0.015045  0.121364  0.105316  0.195587  0.562688  0.009027   
zho_Hans  0.019057  0.102307  0.126379  0.310933  0.441324  0.012036   

model                                             
rating        Fair      Good Very Good Excellent  
lang_src                                          
amh_Ethi  0.237713  0.135406  0.208626  0.288867  
arb_Arab  0.062187  0.071214  0.177533  0.672016  
ben_Beng  0.178536  0.127382  0.221665  0.423270  
deu_Latn  0.016048  0.029087  0.123370  0.825476  
hin_Deva  0.143430  0.115346  0.204614  0.497492  
jpn_Jpan  0.061184  0.118355  0.295888  0.507523  
por_Latn  0.044132  0.044132  0.157472  0.747242  
sat_Olck  0.010030  0.001003       NaN       NaN  
spa_Latn  0.052156  0.066199  0.240722  0.633902  
swh_Latn  0.117352  0.111334  0.194584  0.527583  
tel_Telu  0.154463  0.121364  0.202608  0.468405  
tgl_Latn  0.087262  0.068205  0.166499  0.669007  
zho_Hans  0.056169  0.094283  0.280843  0.556670

In [144]:
print("Assessment version: 66e31bad8e7990f6fd2b6cef5abf4418")

Assessment version: 66e31bad8e7990f6fd2b6cef5abf4418


## Binary comparison

In [ ]:
comparison_data = build_comparison_payload(
    model_kwargs,
    # system_content="You are an expert machine translation evaluator. Your task is to assess the quality of translations by comparing the translated sentence in English with its original English version.\n\nGiven a source text and its translated version, rate the translation quality as either CORRECT or INCORRECT.\n\nYour response must begin with `Rating: ` followed by the rating. Provide no further explanation."
    system_content="""You are an expert machine translation evaluation system, capable of accurately assessing precise matches between original and translated texts. Given an original English sentence and its back-translation into English from another language, assess whether the retranslated sentence accurately conveys the same meaning as the original, ensuring that all facts and details are preserved.\n\nRate the translation quality as either `CORRECT` if the translated sentence is semantically identical to the original, preserving all factual information and details, or `INCORRECT` if it differs in meaning, omits or distorts any facts or details.\n\nRespond with: `Rating: <rating>`. Provide no further explanation."""
)

In [ ]:
comparison_fname = os.path.join(base_dir, "comparison", f"comparison-{comparison_data['prompt_id']}.jsonl")

comparison_batch_input_file = client.files.create(
    file=open(comparison_fname, "rb"),
    purpose="batch"
)

comparison_bobj = client.batches.create(
    input_file_id=comparison_batch_input_file.id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
    "description": f"comparison job for {comparison_batch_input_file.filename}",
    }
)

print(comparison_bobj)
fname = check_store_comparison(comparison_bobj)

In [15]:
fname = os.path.join(base_dir, "comparison", "comparison-7d619c5cb87ef2061ddb2f3c5c0a617c-completed.jsonl")

bin_parsed_comparisons = parse_comparison_response(
    fname,
    valid_ratings=["INCORRECT", "CORRECT"]
)
bin_parsed_comparisons_df = pd.DataFrame(bin_parsed_comparisons)

bin_performance_df = pd.DataFrame(bin_parsed_comparisons)
bin_performance_df = bin_performance_df.groupby(["model", "lang_src", "rating"]).size().unstack(level=[0, 2])
bin_performance_df /= bin_performance_df["gpt-4o"].sum(axis=1).values[0]

print(fname)
# Sort the rating columns in increasing order
# ["INCORRECT", "CORRECT"]
rating_order = ["INCORRECT", "CORRECT"]
cols = [(m, r) for m in models for r in rating_order]

bin_performance_df = bin_performance_df[cols]
bin_performance_df = (
    bin_performance_df
    .loc[[lang_name[name]["code"] for name in ordered_lang_names]]
    .set_index(lang_index.set_index(["Language", "Code"]).index)
)
bin_performance_df

../data/processed/translate-batch/2024-08-31/comparison/comparison-7d619c5cb87ef2061ddb2f3c5c0a617c-completed.jsonl


model                gpt-4-turbo-2024-04-09              gpt-4o          
rating                            INCORRECT   CORRECT INCORRECT   CORRECT
Language        Code                                                     
Nuer            nus                0.998997  0.001003  0.993982  0.006018
Kabiyè          kbp                0.996991  0.003009  0.991976  0.008024
Tamasheq        taq                0.991976  0.008024  0.981946  0.018054
Dzongkha        dzo                1.000000       NaN  0.984955  0.015045
Hindi           hin                0.434303  0.565697  0.392177  0.607823
Bengali         ben                0.526580  0.473420  0.448345  0.551655
Urdu            urd                0.479438  0.520562  0.443330  0.556670
Odia            ory                0.548646  0.451354  0.420261  0.579739
Santhali        sat                1.000000       NaN  0.998997  0.001003
Shan            shn                0.991976  0.008024  0.987964  0.012036
Chinese         zho                0.343029  0.656971  0.304915  0.695085
Standard Arabic arb                0.278837  0.721163  0.234704  0.765296
Spanish         spa                0.198596  0.801404  0.186560  0.813440
French          fra                0.164493  0.835507  0.153460  0.846540

../data/processed/translate-batch/comparison/comparison-ab9c0e8cbe8b4643712926ce3461701e-completed.jsonl


model    gpt-4-turbo-2024-04-09              gpt-4o               gpt-4  \
rating                INCORRECT   CORRECT INCORRECT   CORRECT INCORRECT   
lang_src                                                                  
amh_Ethi               0.789368  0.210632  0.579739  0.420261  0.833501   
arb_Arab               0.260782  0.739218  0.197593  0.802407  0.298897   
ben_Beng               0.472417  0.527583  0.410231  0.589769  0.595787   
deu_Latn               0.088265  0.911735  0.074223  0.925777  0.103310   
hin_Deva               0.427282  0.572718  0.375125  0.624875  0.497492   
jpn_Jpan               0.342026  0.657974  0.283852  0.716148  0.359077   
por_Latn               0.155466  0.844534  0.144433  0.855567  0.170512   
sat_Olck               1.000000       NaN  1.000000       NaN  1.000000   
spa_Latn               0.203611  0.796389  0.176530  0.823470  0.215647   
swh_Latn               0.390171  0.609829  0.328987  0.671013  0.434303   
tel_Telu               0.465396  0.534604  0.385155  0.614845  0.680040   
tgl_Latn               0.236710  0.763290  0.212638  0.787362  0.289870   
zho_Hans               0.281846  0.718154  0.248746  0.751254  0.329990   

model               
rating     CORRECT  
lang_src            
amh_Ethi  0.166499  
arb_Arab  0.701103  
ben_Beng  0.404213  
deu_Latn  0.896690  
hin_Deva  0.502508  
jpn_Jpan  0.640923  
por_Latn  0.829488  
sat_Olck       NaN  
spa_Latn  0.784353  
swh_Latn  0.565697  
tel_Telu  0.319960  
tgl_Latn  0.710130  
zho_Hans  0.670010

In [169]:
print(fname)
# print("Assessment version: df47f7a3577acdd6554e43a374ee67fd")
bin_performance_df = pd.DataFrame(bin_parsed_comparisons)
bin_performance_df = bin_performance_df.groupby(["model", "lang_src", "rating"]).size().unstack(level=[0, 2])
bin_performance_df /= bin_performance_df["gpt-4"].sum(axis=1).values[0]
bin_performance_df

model        gpt-4              gpt-4o          
rating     CORRECT INCORRECT   CORRECT INCORRECT
lang_src                                        
amh_Ethi  0.181545  0.818455  0.462387  0.537613
arb_Arab  0.735206  0.264794  0.829488  0.170512
ben_Beng  0.440321  0.559679  0.629890  0.370110
deu_Latn  0.914744  0.085256  0.938816  0.061184
hin_Deva  0.544634  0.455366  0.683049  0.316951
jpn_Jpan  0.716148  0.283852  0.764293  0.235707
por_Latn  0.864594  0.135406  0.893681  0.106319
sat_Olck       NaN  1.000000       NaN  1.000000
spa_Latn  0.834504  0.165496  0.857573  0.142427
swh_Latn  0.607823  0.392177  0.697091  0.302909
tel_Telu  0.350050  0.649950  0.644935  0.355065
tgl_Latn  0.739218  0.260782  0.805416  0.194584
zho_Hans  0.721163  0.278837  0.795386  0.204614

In [170]:
print(fname)
# print("Assessment version: df47f7a3577acdd6554e43a374ee67fd")
# Sort the rating columns in increasing order
# ["INCORRECT", "CORRECT"]
rating_order = ["INCORRECT", "CORRECT"]
cols = [(m, r) for m in ["gpt-4", "gpt-4o"] for r in rating_order]

bin_performance_df = bin_performance_df[cols]
bin_performance_df

model        gpt-4              gpt-4o          
rating   INCORRECT   CORRECT INCORRECT   CORRECT
lang_src                                        
amh_Ethi  0.818455  0.181545  0.537613  0.462387
arb_Arab  0.264794  0.735206  0.170512  0.829488
ben_Beng  0.559679  0.440321  0.370110  0.629890
deu_Latn  0.085256  0.914744  0.061184  0.938816
hin_Deva  0.455366  0.544634  0.316951  0.683049
jpn_Jpan  0.283852  0.716148  0.235707  0.764293
por_Latn  0.135406  0.864594  0.106319  0.893681
sat_Olck  1.000000       NaN  1.000000       NaN
spa_Latn  0.165496  0.834504  0.142427  0.857573
swh_Latn  0.392177  0.607823  0.302909  0.697091
tel_Telu  0.649950  0.350050  0.355065  0.644935
tgl_Latn  0.260782  0.739218  0.194584  0.805416
zho_Hans  0.278837  0.721163  0.204614  0.795386

In [171]:
# print("Assessment version: df47f7a3577acdd6554e43a374ee67fd")

Assessment version: df47f7a3577acdd6554e43a374ee67fd


## Explain and rate

In [ ]:
comparison_data = build_comparison_payload(
    model_kwargs,
    # system_content="You are an expert machine translation evaluator. Your task is to assess the quality of translations by comparing the translated sentence in English with its original English version.\n\nGiven a source text and its translated version, rate the translation quality as either CORRECT or INCORRECT.\n\nFirst, explain to yourself in one sentence the reason for your rating. Then, end your response with `Rating: ` followed by the rating."
    system_content="""You are an expert machine translation evaluation system, capable of accurately assessing precise matches between original and translated texts. Given an original English sentence and its back-translation into English from another language, assess whether the retranslated sentence accurately conveys the same meaning as the original, ensuring that all facts and details are preserved.\n\nRate the translation quality as either `CORRECT` if the translated sentence is semantically identical to the original, preserving all factual information and details, or `INCORRECT` if it differs in meaning, omits or distorts any facts or details.\n\nFirst, explain to yourself in one sentence the reason for your rating. Then, end your response with `Rating: <rating>`."""
)

assert len(comparison_data["payloads"]) == (997 * (len(lang_name) - 1) * len(models)), len(comparison_data["payloads"]) # 997 is the number of sentences in each language except English  ## 27916

In [49]:
comparison_fname = os.path.join(base_dir, "comparison", f"comparison-{comparison_data['prompt_id']}.jsonl")

comparison_batch_input_file = client.files.create(
    file=open(comparison_fname, "rb"),
    purpose="batch"
)

comparison_bobj = client.batches.create(
    input_file_id=comparison_batch_input_file.id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
    "description": f"comparison job for {comparison_batch_input_file.filename}",
    }
)

print(comparison_bobj)
fname = check_store_comparison(comparison_bobj)

Batch(id='batch_wmajMjyysFsZhpflp9UvSZjW', completion_window='24h', created_at=1725176581, endpoint='/v1/chat/completions', input_file_id='file-l9itqO2DpjDb9lv8AiLNIqdt', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1725262981, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'comparison job for comparison-7eaa633078d0f757ee99aabf1620be09.jsonl'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))
../data/processed/translate-batch/2024-08-31/comparison/comparison-7eaa633078d0f757ee99aabf1620be09-completed.jsonl


In [16]:
fname = os.path.join(base_dir, "comparison", "comparison-7eaa633078d0f757ee99aabf1620be09-completed.jsonl")
print(fname)

explain_parsed_comparisons = parse_comparison_response(
    fname,
    valid_ratings=["INCORRECT", "CORRECT"]
)
explain_parsed_comparisons_df = pd.DataFrame(explain_parsed_comparisons)

explain_performance_df = pd.DataFrame(explain_parsed_comparisons)
explain_performance_df = explain_performance_df.groupby(["model", "lang_src", "rating"]).size().unstack(level=[0, 2])
explain_performance_df /= explain_performance_df["gpt-4o"].sum(axis=1).values[0]

print(fname)
# Sort the rating columns in increasing order
# ["INCORRECT", "CORRECT"]
rating_order = ["INCORRECT", "CORRECT"]
cols = [(m, r) for m in models for r in rating_order]

explain_performance_df = explain_performance_df[cols]
explain_performance_df = (
    explain_performance_df
    .loc[[lang_name[name]["code"] for name in ordered_lang_names]]
    .set_index(lang_index.set_index(["Language", "Code"]).index)
)
explain_performance_df

../data/processed/translate-batch/2024-08-31/comparison/comparison-7eaa633078d0f757ee99aabf1620be09-completed.jsonl
../data/processed/translate-batch/2024-08-31/comparison/comparison-7eaa633078d0f757ee99aabf1620be09-completed.jsonl


model                gpt-4-turbo-2024-04-09              gpt-4o          
rating                            INCORRECT   CORRECT INCORRECT   CORRECT
Language        Code                                                     
Nuer            nus                0.992979  0.007021  0.991976  0.008024
Kabiyè          kbp                0.991976  0.008024  0.984955  0.015045
Tamasheq        taq                0.987964  0.012036  0.970913  0.029087
Dzongkha        dzo                0.998997  0.001003  0.970913  0.029087
Hindi           hin                0.372116  0.627884  0.328987  0.671013
Bengali         ben                0.414243  0.585757  0.362086  0.637914
Urdu            urd                0.390171  0.609829  0.351053  0.648947
Odia            ory                0.466399  0.533601  0.350050  0.649950
Santhali        sat                1.000000       NaN  0.998997  0.001003
Shan            shn                0.983952  0.016048  0.978937  0.021063
Chinese         zho                0.220662  0.779338  0.205617  0.794383
Standard Arabic arb                0.220662  0.779338  0.186560  0.813440
Spanish         spa                0.137412  0.862588  0.120361  0.879639
French          fra                0.117352  0.882648  0.104313  0.895687

In [ ]:
# print("Assessment version: 2ef46e1a0e66a3bbba41278639592a8d")

# LLM alignment

In [17]:
grouped_performance_df = pd.DataFrame(index=performance_df.index, columns=pd.MultiIndex.from_product([models, ["INCORRECT", "CORRECT"]]))
grouped_performance_df

gpt-4-turbo-2024-04-09            gpt-4o        
                                  INCORRECT CORRECT INCORRECT CORRECT
Language        Code                                                 
Nuer            nus                     NaN     NaN       NaN     NaN
Kabiyè          kbp                     NaN     NaN       NaN     NaN
Tamasheq        taq                     NaN     NaN       NaN     NaN
Dzongkha        dzo                     NaN     NaN       NaN     NaN
Hindi           hin                     NaN     NaN       NaN     NaN
Bengali         ben                     NaN     NaN       NaN     NaN
Urdu            urd                     NaN     NaN       NaN     NaN
Odia            ory                     NaN     NaN       NaN     NaN
Santhali        sat                     NaN     NaN       NaN     NaN
Shan            shn                     NaN     NaN       NaN     NaN
Chinese         zho                     NaN     NaN       NaN     NaN
Standard Arabic arb                     NaN     NaN       NaN     NaN
Spanish         spa                     NaN     NaN       NaN     NaN
French          fra                     NaN     NaN       NaN     NaN

In [18]:
grouped_performance_df[("gpt-4-turbo-2024-04-09", "INCORRECT")] = performance_df[
    [("gpt-4-turbo-2024-04-09", "Poor"),
     ("gpt-4-turbo-2024-04-09", "Fair"),
    ("gpt-4-turbo-2024-04-09", "Good")
    ]
].sum(axis=1)

grouped_performance_df[("gpt-4o", "INCORRECT")] = performance_df[
    [("gpt-4o", "Poor"),
     ("gpt-4o", "Fair"),
    ("gpt-4o", "Good")
    ]
].sum(axis=1)

grouped_performance_df[("gpt-4-turbo-2024-04-09", "CORRECT")] = 1 - grouped_performance_df[("gpt-4-turbo-2024-04-09", "INCORRECT")]
grouped_performance_df[("gpt-4o", "CORRECT")] = 1 - grouped_performance_df[("gpt-4o", "INCORRECT")]

grouped_performance_df

gpt-4-turbo-2024-04-09              gpt-4o          
                                  INCORRECT   CORRECT INCORRECT   CORRECT
Language        Code                                                     
Nuer            nus                0.997994  0.002006  0.994985  0.005015
Kabiyè          kbp                0.992979  0.007021  0.987964  0.012036
Tamasheq        taq                0.988967  0.011033  0.975928  0.024072
Dzongkha        dzo                1.000000  0.000000  0.981946  0.018054
Hindi           hin                0.541625  0.458375  0.487462  0.512538
Bengali         ben                0.601805  0.398195  0.534604  0.465396
Urdu            urd                0.557673  0.442327  0.516550  0.483450
Odia            ory                0.622869  0.377131  0.514544  0.485456
Santhali        sat                1.000000  0.000000  1.000000  0.000000
Shan            shn                0.985958  0.014042  0.980943  0.019057
Chinese         zho                0.362086  0.637914  0.319960  0.680040
Standard Arabic arb                0.373119  0.626881  0.311936  0.688064
Spanish         spa                0.304915  0.695085  0.290873  0.709127
French          fra                0.206620  0.793380  0.182548  0.817452

In [19]:
# Compare the two binary rating methods by correlating the results

outcome = "INCORRECT"
outcome = "CORRECT"
test_model = "gpt-4-turbo-2024-04-09"
# test_model = "gpt-4o"
(
    (grouped_performance_df[(test_model, outcome)] - (bin_performance_df[test_model, outcome])).abs().mean(),
    (grouped_performance_df[(test_model, outcome)] - (explain_performance_df[test_model, outcome])).abs().mean(),
)

(0.05090270812437312, 0.09551732119435229)

In [20]:
bin_performance_df

model                gpt-4-turbo-2024-04-09              gpt-4o          
rating                            INCORRECT   CORRECT INCORRECT   CORRECT
Language        Code                                                     
Nuer            nus                0.998997  0.001003  0.993982  0.006018
Kabiyè          kbp                0.996991  0.003009  0.991976  0.008024
Tamasheq        taq                0.991976  0.008024  0.981946  0.018054
Dzongkha        dzo                1.000000       NaN  0.984955  0.015045
Hindi           hin                0.434303  0.565697  0.392177  0.607823
Bengali         ben                0.526580  0.473420  0.448345  0.551655
Urdu            urd                0.479438  0.520562  0.443330  0.556670
Odia            ory                0.548646  0.451354  0.420261  0.579739
Santhali        sat                1.000000       NaN  0.998997  0.001003
Shan            shn                0.991976  0.008024  0.987964  0.012036
Chinese         zho                0.343029  0.656971  0.304915  0.695085
Standard Arabic arb                0.278837  0.721163  0.234704  0.765296
Spanish         spa                0.198596  0.801404  0.186560  0.813440
French          fra                0.164493  0.835507  0.153460  0.846540

In [21]:
explain_performance_df

model                gpt-4-turbo-2024-04-09              gpt-4o          
rating                            INCORRECT   CORRECT INCORRECT   CORRECT
Language        Code                                                     
Nuer            nus                0.992979  0.007021  0.991976  0.008024
Kabiyè          kbp                0.991976  0.008024  0.984955  0.015045
Tamasheq        taq                0.987964  0.012036  0.970913  0.029087
Dzongkha        dzo                0.998997  0.001003  0.970913  0.029087
Hindi           hin                0.372116  0.627884  0.328987  0.671013
Bengali         ben                0.414243  0.585757  0.362086  0.637914
Urdu            urd                0.390171  0.609829  0.351053  0.648947
Odia            ory                0.466399  0.533601  0.350050  0.649950
Santhali        sat                1.000000       NaN  0.998997  0.001003
Shan            shn                0.983952  0.016048  0.978937  0.021063
Chinese         zho                0.220662  0.779338  0.205617  0.794383
Standard Arabic arb                0.220662  0.779338  0.186560  0.813440
Spanish         spa                0.137412  0.862588  0.120361  0.879639
French          fra                0.117352  0.882648  0.104313  0.895687

In [22]:
explain_parsed_comparisons_df.index = (
    explain_parsed_comparisons_df["lang_src"] +
    "-eng_Latn" + "-" +
    explain_parsed_comparisons_df["idx"].astype(str).str.zfill(3) + "-" +
    explain_parsed_comparisons_df["model"]
)
explain_parsed_comparisons_df

,idx,lang_src,model,rating
arb_Arab-eng_Latn-000-gpt-4-turbo-2024-04-09,0,arb_Arab,gpt-4-turbo-2024-04-09,CORRECT
arb_Arab-eng_Latn-001-gpt-4-turbo-2024-04-09,1,arb_Arab,gpt-4-turbo-2024-04-09,CORRECT
arb_Arab-eng_Latn-002-gpt-4-turbo-2024-04-09,2,arb_Arab,gpt-4-turbo-2024-04-09,CORRECT
arb_Arab-eng_Latn-003-gpt-4-turbo-2024-04-09,3,arb_Arab,gpt-4-turbo-2024-04-09,INCORRECT
arb_Arab-eng_Latn-004-gpt-4-turbo-2024-04-09,4,arb_Arab,gpt-4-turbo-2024-04-09,INCORRECT
...,...,...,...,...
zho_Hans-eng_Latn-992-gpt-4o,992,zho_Hans,gpt-4o,CORRECT
zho_Hans-eng_Latn-993-gpt-4o,993,zho_Hans,gpt-4o,INCORRECT
zho_Hans-eng_Latn-994-gpt-4o,994,zho_Hans,gpt-4o,CORRECT
zho_Hans-eng_Latn-995-gpt-4o,995,zho_Hans,gpt-4o,CORRECT


In [23]:
bin_parsed_comparisons_df.index = (
    bin_parsed_comparisons_df["lang_src"] +
    "-eng_Latn" + "-" +
    bin_parsed_comparisons_df["idx"].astype(str).str.zfill(3) + "-" +
    bin_parsed_comparisons_df["model"]
)
bin_parsed_comparisons_df

,idx,lang_src,model,rating
arb_Arab-eng_Latn-000-gpt-4-turbo-2024-04-09,0,arb_Arab,gpt-4-turbo-2024-04-09,CORRECT
arb_Arab-eng_Latn-001-gpt-4-turbo-2024-04-09,1,arb_Arab,gpt-4-turbo-2024-04-09,CORRECT
arb_Arab-eng_Latn-002-gpt-4-turbo-2024-04-09,2,arb_Arab,gpt-4-turbo-2024-04-09,CORRECT
arb_Arab-eng_Latn-003-gpt-4-turbo-2024-04-09,3,arb_Arab,gpt-4-turbo-2024-04-09,INCORRECT
arb_Arab-eng_Latn-004-gpt-4-turbo-2024-04-09,4,arb_Arab,gpt-4-turbo-2024-04-09,INCORRECT
...,...,...,...,...
zho_Hans-eng_Latn-992-gpt-4o,992,zho_Hans,gpt-4o,CORRECT
zho_Hans-eng_Latn-993-gpt-4o,993,zho_Hans,gpt-4o,INCORRECT
zho_Hans-eng_Latn-994-gpt-4o,994,zho_Hans,gpt-4o,CORRECT
zho_Hans-eng_Latn-995-gpt-4o,995,zho_Hans,gpt-4o,CORRECT


In [24]:
bin_disagreements = bin_parsed_comparisons_df[bin_parsed_comparisons_df["rating"] != explain_parsed_comparisons_df["rating"]]
bin_disagreements

,idx,lang_src,model,rating
arb_Arab-eng_Latn-007-gpt-4-turbo-2024-04-09,7,arb_Arab,gpt-4-turbo-2024-04-09,INCORRECT
arb_Arab-eng_Latn-017-gpt-4-turbo-2024-04-09,17,arb_Arab,gpt-4-turbo-2024-04-09,INCORRECT
arb_Arab-eng_Latn-023-gpt-4-turbo-2024-04-09,23,arb_Arab,gpt-4-turbo-2024-04-09,INCORRECT
arb_Arab-eng_Latn-027-gpt-4-turbo-2024-04-09,27,arb_Arab,gpt-4-turbo-2024-04-09,INCORRECT
arb_Arab-eng_Latn-038-gpt-4-turbo-2024-04-09,38,arb_Arab,gpt-4-turbo-2024-04-09,INCORRECT
...,...,...,...,...
zho_Hans-eng_Latn-922-gpt-4o,922,zho_Hans,gpt-4o,INCORRECT
zho_Hans-eng_Latn-935-gpt-4o,935,zho_Hans,gpt-4o,INCORRECT
zho_Hans-eng_Latn-943-gpt-4o,943,zho_Hans,gpt-4o,INCORRECT
zho_Hans-eng_Latn-959-gpt-4o,959,zho_Hans,gpt-4o,CORRECT


In [25]:
sampled_bin_disagreements = (
    bin_disagreements
    .groupby(["lang_src", "model", "rating"])[["idx", "lang_src", "model", "rating"]]
    .apply(lambda x: x.sample(min(5, len(x)), random_state=1029))
)

sampled_bin_disagreements

idx  \
lang_src model                  rating                                                        
arb_Arab gpt-4-turbo-2024-04-09 CORRECT   arb_Arab-eng_Latn-181-gpt-4-turbo-2024-04-09  181   
                                          arb_Arab-eng_Latn-699-gpt-4-turbo-2024-04-09  699   
                                          arb_Arab-eng_Latn-049-gpt-4-turbo-2024-04-09   49   
                                          arb_Arab-eng_Latn-599-gpt-4-turbo-2024-04-09  599   
                                          arb_Arab-eng_Latn-751-gpt-4-turbo-2024-04-09  751   
...                                                                                     ...   
zho_Hans gpt-4o                 INCORRECT zho_Hans-eng_Latn-040-gpt-4o                   40   
                                          zho_Hans-eng_Latn-543-gpt-4o                  543   
                                          zho_Hans-eng_Latn-897-gpt-4o                  897   
                                          zho_Hans-eng_Latn-193-gpt-4o                  193   
                                          zho_Hans-eng_Latn-527-gpt-4o                  527   

                                                                                        lang_src  \
lang_src model                  rating                                                             
arb_Arab gpt-4-turbo-2024-04-09 CORRECT   arb_Arab-eng_Latn-181-gpt-4-turbo-2024-04-09  arb_Arab   
                                          arb_Arab-eng_Latn-699-gpt-4-turbo-2024-04-09  arb_Arab   
                                          arb_Arab-eng_Latn-049-gpt-4-turbo-2024-04-09  arb_Arab   
                                          arb_Arab-eng_Latn-599-gpt-4-turbo-2024-04-09  arb_Arab   
                                          arb_Arab-eng_Latn-751-gpt-4-turbo-2024-04-09  arb_Arab   
...                                                                                          ...   
zho_Hans gpt-4o                 INCORRECT zho_Hans-eng_Latn-040-gpt-4o                  zho_Hans   
                                          zho_Hans-eng_Latn-543-gpt-4o                  zho_Hans   
                                          zho_Hans-eng_Latn-897-gpt-4o                  zho_Hans   
                                          zho_Hans-eng_Latn-193-gpt-4o                  zho_Hans   
                                          zho_Hans-eng_Latn-527-gpt-4o                  zho_Hans   

                                                                                                         model  \
lang_src model                  rating                                                                           
arb_Arab gpt-4-turbo-2024-04-09 CORRECT   arb_Arab-eng_Latn-181-gpt-4-turbo-2024-04-09  gpt-4-turbo-2024-04-09   
                                          arb_Arab-eng_Latn-699-gpt-4-turbo-2024-04-09  gpt-4-turbo-2024-04-09   
                                          arb_Arab-eng_Latn-049-gpt-4-turbo-2024-04-09  gpt-4-turbo-2024-04-09   
                                          arb_Arab-eng_Latn-599-gpt-4-turbo-2024-04-09  gpt-4-turbo-2024-04-09   
                                          arb_Arab-eng_Latn-751-gpt-4-turbo-2024-04-09  gpt-4-turbo-2024-04-09   
...                                                                                                        ...   
zho_Hans gpt-4o                 INCORRECT zho_Hans-eng_Latn-040-gpt-4o                                  gpt-4o   
                                          zho_Hans-eng_Latn-543-gpt-4o                                  gpt-4o   
                                          zho_Hans-eng_Latn-897-gpt-4o                                  gpt-4o   
                                          zho_Hans-eng_Latn-193-gpt-4o                                  gpt-4o   
                                          zho_Hans-eng_Latn-527-gpt-4o                                  gpt-4o   

                                                                    

In [20]:
# Load the translated sentences from the compiled files
# Any file can be used as they all contain the same sentences.
comparison_file = os.path.join(base_dir, "comparison", "comparison-7eaa633078d0f757ee99aabf1620be09.jsonl")

with open(comparison_file, "r") as f:
    comparison_responses = [json.loads(line) for idx, line in enumerate(f)]
    comparison_responses = {
        "-".join(data["custom_id"].split("-")[2:-1]): data["body"]["messages"][1]["content"] for data in comparison_responses
    }

print(comparison_responses["arb_Arab-eng_Latn-017-gpt-4-turbo-2024-04-09"])

Source: British newspaper The Guardian suggested Deutsche Bank controlled roughly a third of the 1200 shell companies used to accomplish this.

Translated: The British newspaper The Guardian indicated that "Deutsche Bank" controls nearly a third of 1200 fictitious companies used to achieve that.



In [21]:
# Compile the list of sentences for the sampled disagreements
sampled_disagreement_contents = []

for idx, row in sampled_bin_disagreements.iterrows():
    sampled_disagreement_contents.append(
        dict(
            lang_src=row["lang_src"],
            model=row["model"],
            bin_rating=bin_parsed_comparisons_df.loc[idx[-1], "rating"],
            explain_rating=explain_parsed_comparisons_df.loc[idx[-1], "rating"],
            idx=row["idx"],
            content=comparison_responses[idx[-1]]
        )
    )

sampled_disagreement_contents_df = pd.DataFrame(sampled_disagreement_contents)

# Extract the source and target sentences
sampled_disagreement_contents_df["source"] = sampled_disagreement_contents_df["content"].str.split("\n\n").str[0]

sampled_disagreement_contents_df["target"] = sampled_disagreement_contents_df["content"].str.split("\n\n").str[1]

# Drop the content column
sampled_disagreement_contents_df.drop(columns=["content"], inplace=True)

# Save the sampled disagreements between the binary and explainable ratings to a file
sampled_disagreement_contents_df.to_excel("sampled_disagreement_contents.xlsx", index=False)

sampled_disagreement_contents_df

,lang_src,model,bin_rating,explain_rating,idx,source,target
0,arb_Arab,gpt-4-turbo-2024-04-09,CORRECT,INCORRECT,181,Source: The Irish government is stressing the ...,Translated: The Irish government also emphasiz...
1,arb_Arab,gpt-4-turbo-2024-04-09,CORRECT,INCORRECT,699,Source: This letter must be legalized by the E...,Translated: This letter must be authenticated ...
2,arb_Arab,gpt-4-turbo-2024-04-09,CORRECT,INCORRECT,49,Source: The New Zealand police had trouble usi...,Translated: The New Zealand police faced a pro...
3,arb_Arab,gpt-4-turbo-2024-04-09,CORRECT,INCORRECT,599,Source: They looked like rooms. He was the fir...,Translated: He was the first person to notice ...
4,arb_Arab,gpt-4-turbo-2024-04-09,CORRECT,INCORRECT,751,Source: Setting up a tent on private property ...,Translated: Setting up a tent on private prope...
...,...,...,...,...,...,...,...
184,zho_Hans,gpt-4o,INCORRECT,CORRECT,40,"Source: John Grant, from WNED Buffalo (Reading...",Translated: John Grant from Buffalo's WNED sta...
185,zho_Hans,gpt-4o,INCORRECT,CORRECT,543,Source: Gridley or Stark placed a stake about ...,Translated: Gridley or Stark placed a wooden s...
186,zho_Hans,gpt-4o,INCORRECT,CORRECT,897,Source: Some boots have studs and there is stu...,Translated: Some boots have cleats; there are ...
187,zho_Hans,gpt-4o,INCORRECT,CORRECT,193,Source: Schumacher who retired in 2006 after w...,Translated: After winning the Formula 1 World ...


In [145]:
# arb_Arab-eng_Latn-007-gpt-4-turbo-2024-04-09
print("""Source: The protest started around 11:00 local time (UTC+1) on Whitehall opposite the police-guarded entrance to Downing Street, the Prime Minister's official residence.\n\nTranslated: The protest began around 11:00 AM local time (UTC+1) at \"Whitehall\" in the direction opposite to \"Downing Street,\" which is guarded by the police and is the official residence of the Prime Minister.\n""")

Source: The protest started around 11:00 local time (UTC+1) on Whitehall opposite the police-guarded entrance to Downing Street, the Prime Minister's official residence.

Translated: The protest began around 11:00 AM local time (UTC+1) at "Whitehall" in the direction opposite to "Downing Street," which is guarded by the police and is the official residence of the Prime Minister.



In [146]:
# arb_Arab-eng_Latn-017-gpt-4-turbo-2024-04-09
print("""Source: British newspaper The Guardian suggested Deutsche Bank controlled roughly a third of the 1200 shell companies used to accomplish this.\n\nTranslated: The British newspaper The Guardian indicated that \"Deutsche Bank\" controls nearly a third of 1200 fictitious companies used to achieve that.\n""")

Source: British newspaper The Guardian suggested Deutsche Bank controlled roughly a third of the 1200 shell companies used to accomplish this.

Translated: The British newspaper The Guardian indicated that "Deutsche Bank" controls nearly a third of 1200 fictitious companies used to achieve that.



In [22]:
performance_df

model                gpt-4-turbo-2024-04-09                                \
rating                                 Poor      Fair      Good Very Good   
Language        Code                                                        
Nuer            nus                0.943831  0.031093  0.023069  0.002006   
Kabiyè          kbp                0.927783  0.036108  0.029087  0.004012   
Tamasheq        taq                0.890672  0.054162  0.044132  0.006018   
Dzongkha        dzo                0.978937  0.017051  0.004012       NaN   
Hindi           hin                0.036108  0.083250  0.422267  0.343029   
Bengali         ben                0.048144  0.105316  0.448345  0.277834   
Urdu            urd                0.046138  0.093280  0.418255  0.315948   
Odia            ory                0.096289  0.110331  0.416249  0.265797   
Santhali        sat                0.998997  0.001003       NaN       NaN   
Shan            shn                0.894684  0.049147  0.042126  0.010030   
Chinese         zho                0.011033  0.030090  0.320963  0.458375   
Standard Arabic arb                0.020060  0.047141  0.305918  0.499498   
Spanish         spa                0.007021  0.022066  0.275827  0.539619   
French          fra                0.007021  0.016048  0.183551  0.662989   

model                             gpt-4o                                \
rating               Excellent      Poor      Fair      Good Very Good   
Language        Code                                                     
Nuer            nus        NaN  0.939819  0.026078  0.029087  0.003009   
Kabiyè          kbp   0.003009  0.894684  0.053159  0.040120  0.006018   
Tamasheq        taq   0.005015  0.845537  0.077232  0.053159  0.017051   
Dzongkha        dzo        NaN  0.787362  0.119358  0.075226  0.011033   
Hindi           hin   0.115346  0.032096  0.063190  0.392177  0.367101   
Bengali         ben   0.120361  0.037111  0.092277  0.405216  0.342026   
Urdu            urd   0.126379  0.040120  0.083250  0.393180  0.359077   
Odia            ory   0.111334  0.054162  0.089268  0.371113  0.351053   
Santhali        sat        NaN  0.990973  0.006018  0.003009       NaN   
Shan            shn   0.004012  0.879639  0.059178  0.042126  0.013039   
Chinese         zho   0.179539  0.008024  0.025075  0.286861  0.499498   
Standard Arabic arb   0.127382  0.011033  0.023069  0.277834  0.548646   
Spanish         spa   0.155466  0.002006  0.025075  0.263791  0.549649   
French          fra   0.130391  0.006018  0.016048  0.160481  0.667001   

model                           
rating               Excellent  
Language        Code            
Nuer            nus   0.002006  
Kabiyè          kbp   0.006018  
Tamasheq        taq   0.007021  
Dzongkha        dzo   0.007021  
Hindi           hin   0.145436  
Bengali         ben   0.123370  
Urdu            urd   0.124373  
Odia            ory   0.134403  
Santhali        sat        NaN  
Shan            shn   0.006018  
Chinese         zho   0.180542  
Standard Arabic arb   0.139418  
Spanish         spa   0.159478  
French          fra   0.150451

In [24]:
bin_performance_df

model                gpt-4-turbo-2024-04-09              gpt-4o          
rating                            INCORRECT   CORRECT INCORRECT   CORRECT
Language        Code                                                     
Nuer            nus                0.998997  0.001003  0.993982  0.006018
Kabiyè          kbp                0.996991  0.003009  0.991976  0.008024
Tamasheq        taq                0.991976  0.008024  0.981946  0.018054
Dzongkha        dzo                1.000000       NaN  0.984955  0.015045
Hindi           hin                0.434303  0.565697  0.392177  0.607823
Bengali         ben                0.526580  0.473420  0.448345  0.551655
Urdu            urd                0.479438  0.520562  0.443330  0.556670
Odia            ory                0.548646  0.451354  0.420261  0.579739
Santhali        sat                1.000000       NaN  0.998997  0.001003
Shan            shn                0.991976  0.008024  0.987964  0.012036
Chinese         zho                0.343029  0.656971  0.304915  0.695085
Standard Arabic arb                0.278837  0.721163  0.234704  0.765296
Spanish         spa                0.198596  0.801404  0.186560  0.813440
French          fra                0.164493  0.835507  0.153460  0.846540

In [23]:
grouped_performance_df

gpt-4-turbo-2024-04-09              gpt-4o          
                                  INCORRECT   CORRECT INCORRECT   CORRECT
Language        Code                                                     
Nuer            nus                0.997994  0.002006  0.994985  0.005015
Kabiyè          kbp                0.992979  0.007021  0.987964  0.012036
Tamasheq        taq                0.988967  0.011033  0.975928  0.024072
Dzongkha        dzo                1.000000  0.000000  0.981946  0.018054
Hindi           hin                0.541625  0.458375  0.487462  0.512538
Bengali         ben                0.601805  0.398195  0.534604  0.465396
Urdu            urd                0.557673  0.442327  0.516550  0.483450
Odia            ory                0.622869  0.377131  0.514544  0.485456
Santhali        sat                1.000000  0.000000  1.000000  0.000000
Shan            shn                0.985958  0.014042  0.980943  0.019057
Chinese         zho                0.362086  0.637914  0.319960  0.680040
Standard Arabic arb                0.373119  0.626881  0.311936  0.688064
Spanish         spa                0.304915  0.695085  0.290873  0.709127
French          fra                0.206620  0.793380  0.182548  0.817452

In [27]:
parsed_comparisons_df.index = parsed_comparisons_df["lang_src"] + "-eng_Latn" + "-" + parsed_comparisons_df["idx"].astype(str).str.zfill(3) + "-" + parsed_comparisons_df["model"]
parsed_comparisons_df

,idx,lang_src,model,rating
arb_Arab-eng_Latn-000-gpt-4-turbo-2024-04-09,0,arb_Arab,gpt-4-turbo-2024-04-09,Very Good
arb_Arab-eng_Latn-001-gpt-4-turbo-2024-04-09,1,arb_Arab,gpt-4-turbo-2024-04-09,Very Good
arb_Arab-eng_Latn-002-gpt-4-turbo-2024-04-09,2,arb_Arab,gpt-4-turbo-2024-04-09,Very Good
arb_Arab-eng_Latn-003-gpt-4-turbo-2024-04-09,3,arb_Arab,gpt-4-turbo-2024-04-09,Fair
arb_Arab-eng_Latn-004-gpt-4-turbo-2024-04-09,4,arb_Arab,gpt-4-turbo-2024-04-09,Good
...,...,...,...,...
taq_Latn-eng_Latn-992-gpt-4-turbo-2024-04-09,992,taq_Latn,gpt-4-turbo-2024-04-09,Poor
taq_Latn-eng_Latn-993-gpt-4-turbo-2024-04-09,993,taq_Latn,gpt-4-turbo-2024-04-09,Poor
taq_Latn-eng_Latn-994-gpt-4-turbo-2024-04-09,994,taq_Latn,gpt-4-turbo-2024-04-09,Poor
taq_Latn-eng_Latn-995-gpt-4-turbo-2024-04-09,995,taq_Latn,gpt-4-turbo-2024-04-09,Poor


In [28]:
full_comparisons_df = parsed_comparisons_df.merge(bin_parsed_comparisons_df[["rating"]], left_index=True, right_index=True, suffixes=("", "_bin")).merge(explain_parsed_comparisons_df[["rating"]], left_index=True, right_index=True, suffixes=("", "_explain"))

full_comparisons_df

,idx,lang_src,model,rating,rating_bin,rating_explain
arb_Arab-eng_Latn-000-gpt-4-turbo-2024-04-09,0,arb_Arab,gpt-4-turbo-2024-04-09,Very Good,CORRECT,CORRECT
arb_Arab-eng_Latn-001-gpt-4-turbo-2024-04-09,1,arb_Arab,gpt-4-turbo-2024-04-09,Very Good,CORRECT,CORRECT
arb_Arab-eng_Latn-002-gpt-4-turbo-2024-04-09,2,arb_Arab,gpt-4-turbo-2024-04-09,Very Good,CORRECT,CORRECT
arb_Arab-eng_Latn-003-gpt-4-turbo-2024-04-09,3,arb_Arab,gpt-4-turbo-2024-04-09,Fair,INCORRECT,INCORRECT
arb_Arab-eng_Latn-004-gpt-4-turbo-2024-04-09,4,arb_Arab,gpt-4-turbo-2024-04-09,Good,INCORRECT,INCORRECT
...,...,...,...,...,...,...
taq_Latn-eng_Latn-992-gpt-4-turbo-2024-04-09,992,taq_Latn,gpt-4-turbo-2024-04-09,Poor,INCORRECT,INCORRECT
taq_Latn-eng_Latn-993-gpt-4-turbo-2024-04-09,993,taq_Latn,gpt-4-turbo-2024-04-09,Poor,INCORRECT,INCORRECT
taq_Latn-eng_Latn-994-gpt-4-turbo-2024-04-09,994,taq_Latn,gpt-4-turbo-2024-04-09,Poor,INCORRECT,INCORRECT
taq_Latn-eng_Latn-995-gpt-4-turbo-2024-04-09,995,taq_Latn,gpt-4-turbo-2024-04-09,Poor,INCORRECT,INCORRECT


In [29]:
full_comparisons_df.pivot_table(index=["model", "rating"], columns=["model", "rating_bin"], aggfunc="size", fill_value=0).loc[[(i, j) for i in ["gpt-4-turbo-2024-04-09", "gpt-4o"] for j in ["Excellent", "Very Good", "Good", "Fair", "Poor"]]] #[["INCORRECT", "CORRECT"]]

model                            gpt-4-turbo-2024-04-09            gpt-4o  \
rating_bin                                      CORRECT INCORRECT CORRECT   
model                  rating                                               
gpt-4-turbo-2024-04-09 Excellent                   1033        42       0   
                       Very Good                   3155       220       0   
                       Good                         843      2082       0   
                       Fair                           0       694       0   
                       Poor                           0      5889       0   
gpt-4o                 Excellent                      0         0    1139   
                       Very Good                      0         0    3476   
                       Good                           0         0     843   
                       Fair                           0         0       1   
                       Poor                           0         0       1   

model                                       
rating_bin                       INCORRECT  
model                  rating               
gpt-4-turbo-2024-04-09 Excellent         0  
                       Very Good         0  
                       Good              0  
                       Fair              0  
                       Poor              0  
gpt-4o                 Excellent        43  
                       Very Good       247  
                       Good           1942  
                       Fair            755  
                       Poor           5511

In [30]:
# Define the model and rating combinations for indexing
model_rating_index = pd.MultiIndex.from_product(
    [["gpt-4-turbo-2024-04-09", "gpt-4o"], ["Excellent", "Very Good", "Good", "Fair", "Poor"]],
    names=["model", "rating"]
)

# Define the model and rating_explain combinations for columns
model_rating_columns = pd.MultiIndex.from_product(
    [["gpt-4-turbo-2024-04-09", "gpt-4o"], ["INCORRECT", "CORRECT"]],
    names=["model", "rating_explain"]
)

# Now create the pivot table and apply the multi-index selections
pivot_table = full_comparisons_df.pivot_table(
    index=["model", "rating"],
    columns=["model", "rating_explain"],
    aggfunc="size",
    fill_value=0
)

# Apply the selections using .loc with predefined indices
filtered_table = pivot_table.loc[model_rating_index, model_rating_columns]

# Display or return the filtered table
filtered_table


model                            gpt-4-turbo-2024-04-09            gpt-4o  \
rating_explain                                INCORRECT CORRECT INCORRECT   
model                  rating                                               
gpt-4-turbo-2024-04-09 Excellent                     14    1061         0   
                       Very Good                     83    3292         0   
                       Good                        1594    1331         0   
                       Fair                         690       4         0   
                       Poor                        5889       0         0   
gpt-4o                 Excellent                      0       0        15   
                       Very Good                      0       0        99   
                       Good                           0       0      1505   
                       Fair                           0       0       751   
                       Poor                           0       0      5512   

model                                     
rating_explain                   CORRECT  
model                  rating             
gpt-4-turbo-2024-04-09 Excellent       0  
                       Very Good       0  
                       Good            0  
                       Fair            0  
                       Poor            0  
gpt-4o                 Excellent    1167  
                       Very Good    3624  
                       Good         1280  
                       Fair            5  
                       Poor            0

In [51]:
def get_model_concordance(model: str, full_comparisons_df: pd.DataFrame, verbose: bool = False) -> dict:
    """
    Get the concordance between the binary and explainable ratings for a given model, relative to the multi-scale ratings.
    """
    # Filter the full comparisons DataFrame for the given model
    model_comparisons_df = full_comparisons_df[full_comparisons_df["model"] == model]

    # Generate the first pivot table for rating_bin
    pivot_bin = model_comparisons_df.pivot_table(
        index=["rating"],
        columns=["rating_bin"],
        aggfunc="size",
        fill_value=0
    ).loc[["Excellent", "Very Good", "Good", "Fair", "Poor"]][["INCORRECT", "CORRECT"]]

    # Generate the second pivot table for rating_explain
    pivot_explain = model_comparisons_df.pivot_table(
        index=["rating"],
        columns=["rating_explain"],
        aggfunc="size",
        fill_value=0
    ).loc[["Excellent", "Very Good", "Good", "Fair", "Poor"]][["INCORRECT", "CORRECT"]]

    # Combine the two pivot tables using multi-index columns
    combined_pivot = pd.concat([pivot_bin, pivot_explain], axis=1, keys=["rating_bin", "rating_explain"])

    # Multiply the values by 2 since we divided the whole row by the sum of the row.
    combined_pivot_pct = (2 * combined_pivot.div(combined_pivot.sum(axis=1), axis=0) * 100).round(2)

    if verbose:
        # Display the resulting table
        print(combined_pivot)
        print(combined_pivot_pct)

    return dict(combined_pivot=combined_pivot, combined_pivot_pct=combined_pivot_pct)

In [52]:
gpt_4o_concordance = get_model_concordance("gpt-4o", full_comparisons_df)
gpt_4o_concordance["combined_pivot_pct"]

rating_bin         rating_explain        
rating_bin  INCORRECT CORRECT      INCORRECT CORRECT
rating                                              
Excellent        3.64   96.36           1.27   98.73
Very Good        6.63   93.37           2.66   97.34
Good            69.73   30.27          54.04   45.96
Fair            99.87    0.13          99.34    0.66
Poor            99.98    0.02         100.00    0.00

In [53]:
gpt_4_turbo_concordance = get_model_concordance("gpt-4-turbo-2024-04-09", full_comparisons_df)
gpt_4_turbo_concordance["combined_pivot_pct"]

rating_bin         rating_explain        
rating_bin  INCORRECT CORRECT      INCORRECT CORRECT
rating                                              
Excellent        3.91   96.09           1.30   98.70
Very Good        6.52   93.48           2.46   97.54
Good            71.18   28.82          54.50   45.50
Fair           100.00    0.00          99.42    0.58
Poor           100.00    0.00         100.00    0.00

In [ ]:
gpt_4o_concordance["combined_pivot_pct"]

# The Double Jeopardy

Merge the tokenization fragmentation and the back-translation tasks performance to visualize the jeopardy.

In [58]:
explain_performance_df.index

MultiIndex([(           'Nuer', 'nus'),
            (         'Kabiyè', 'kbp'),
            (       'Tamasheq', 'taq'),
            (       'Dzongkha', 'dzo'),
            (          'Hindi', 'hin'),
            (        'Bengali', 'ben'),
            (           'Urdu', 'urd'),
            (           'Odia', 'ory'),
            (       'Santhali', 'sat'),
            (           'Shan', 'shn'),
            (        'Chinese', 'zho'),
            ('Standard Arabic', 'arb'),
            (        'Spanish', 'spa'),
            (         'French', 'fra')],
           names=['Language', 'Code'])

In [57]:
from pathlib import Path

DATA_DIR = Path("../data/processed")


premium_df = pd.read_csv(DATA_DIR / "lang_pop2022adj_costs.csv")
cost = premium_df.drop_duplicates(subset=["lang"])
cost

,lang,location,country,year,population,population_2022adj,NY.GDP.PCAP.CD,id,incomeLevel,cl100k_base,o200k_base
0,ace,Indonesia,IDN,2010.0,2.840000e+06,3.244518e+06,4787.999308,IDN,Upper middle income,2.910664,2.235000
1,acm,United Arab Emirates,ARE,2020.0,3.800000e+04,3.894300e+04,53707.980081,ARE,High income,3.000731,1.389443
26,acq,United Arab Emirates,ARE,2020.0,6.000000e+04,6.148800e+04,53707.980081,ARE,High income,3.031674,1.401661
42,aeb,Belgium,BEL,2020.0,2.090000e+04,2.125700e+04,49926.825430,BEL,High income,2.935153,1.431176
53,afr,Angola,AGO,2023.0,2.400000e+03,2.400000e+03,3000.444231,AGO,Lower middle income,1.705142,1.437214
...,...,...,...,...,...,...,...,...,...,...,...
2969,ydd,Belarus,BLR,2017.0,7.000000e+03,6.821000e+03,7888.263711,BLR,Upper middle income,5.602995,2.033419
2983,yor,Australia,AUS,2021.0,4.020000e+03,4.076000e+03,65099.845912,AUS,High income,2.984898,2.186992
3001,yue,Australia,AUS,2021.0,2.950000e+05,2.990750e+05,65099.845912,AUS,High income,2.111599,1.390468
3036,zho,China,CHN,NaN,1.352665e+09,1.352665e+09,12720.216318,CHN,Upper middle income,2.019078,1.339837


In [85]:
# import seaborn as sns
import plotly.express as px

In [136]:
model_id_map = {
    "gpt-4-turbo-2024-04-09": "GPT-4",
    "gpt-4o": "GPT-4o"
}
combined_model_data = []
for model in ["gpt-4o", "gpt-4-turbo-2024-04-09"]:

    model_data = explain_performance_df[model].reset_index().merge(cost[["lang", "cl100k_base", "o200k_base"]], left_on="Code", right_on="lang")
    model_data = pd.concat([model_data, pd.DataFrame([{"Language": "English", "Code": "eng", "INCORRECT": 0, "CORRECT": 1, "lang": "eng", "cl100k_base": 1, "o200k_base": 1}])], axis=0)

    cl100_model_data = model_data[["Language", "Code", "CORRECT", "cl100k_base"]].copy().rename(columns={"cl100k_base": "Fragmentation"})
    cl200_model_data = model_data[["Language", "Code", "CORRECT", "o200k_base"]].copy().rename(columns={"o200k_base": "Fragmentation"})

    cl100_model_data["Tokenizer"] = "GPT-4 (cl100k_base)"
    cl200_model_data["Tokenizer"] = "GPT-4o (o200k_base)"


    _combined_model_data = pd.concat([cl100_model_data, cl200_model_data], axis=0)
    _combined_model_data["Performance"] = _combined_model_data["CORRECT"] * 100

    _combined_model_data["Back-translation model"] = model_id_map[model]
    combined_model_data.append(_combined_model_data)

combined_model_data = pd.concat(combined_model_data, axis=0)


In [139]:
fig = px.scatter(
    combined_model_data,
    x="Performance",
    y="Fragmentation",
    color="Tokenizer",
    symbol="Tokenizer",
    text="Language",
    facet_row="Back-translation model",
    trendline="ols",
)

fig.update_layout(
    plot_bgcolor='white',
    title_text='Double Jeopardy in Large Language Models (LLMs)', # title of plot
    yaxis_title_text='Cost (fragmentation)', # xaxis label
    xaxis_title_text='LLM performance (back-translation)', # yaxis label
    font=dict(size=10), # font size
#     bargap=0.01, # gap between bars of adjacent location coordinates
    width=600,
    height=800,
    legend=dict(
        orientation="v",  # Keep the legend vertical
        x=0.86,  # Move it inside the plot area on the right
        y=1,
        xanchor="center",
        yanchor="top",
        bgcolor="rgba(255,255,255,0.5)"  # Optional: add a semi-transparent background to the legend
    ),    # Make sure the axis lines are visible and customize their appearance
    xaxis=dict(
        showline=True,  # Show the x-axis line
        linewidth=1,    # Set the width of the x-axis line
        linecolor='#cfcfcf'  # Set the color of the x-axis line
    ),
    yaxis=dict(
        showline=True,  # Show the y-axis line
        linewidth=1,    # Set the width of the y-axis line
        linecolor='#cfcfcf'  # Set the color of the y-axis line
    ),
    xaxis2=dict(
        showline=True,  # Show the x-axis line
        linewidth=1,    # Set the width of the x-axis line
        linecolor='#cfcfcf'  # Set the color of the x-axis line
    ),
    yaxis2=dict(
        showline=True,  # Show the y-axis line
        linewidth=1,    # Set the width of the y-axis line
        linecolor='#cfcfcf'  # Set the color of the y-axis line
    )

)
fig.update_traces(textposition="middle right")

fig.update_yaxes(
    row=1, col=1,
    title_text='Fragmentation (Cost)',
    # linecolor='#afafaf',
    # linewidth=1,
    # showline=True,
)
fig.update_yaxes(title_text='Fragmentation (Cost)', row=2, col=1)

fig.write_image("../reports/double-jeopardy-llm.png", format="png", scale=4)
fig.show()